In [1]:
%matplotlib widget

In [2]:
import numpy as np
import h5py
import pandas as pd
from scipy.integrate import cumtrapz
from scipy.signal import butter, sosfiltfilt, detrend
from pathlib import Path
import matplotlib.pyplot as plt

In [3]:
gait_sets_path = Path('/home/lukasadamowicz/Documents/Datasets/processed')

subj_files = [i for i in (gait_sets_path / 'daliac').glob('**/*')]

with h5py.File(subj_files[0], 'r') as f:
    fs = f['stairs-ascending']['Trial 1'].attrs.get('Sampling rate')
    stair_acc = f['stairs-ascending']['Trial 1']['Accelerometer'][()]
    walk_acc = f['walking']['Trial 1']['Accelerometer'][()]

stair_t = np.arange(0, stair_acc.shape[0] / fs, 1 / fs)
walk_t = np.arange(0, walk_acc.shape[0] / fs, 1 / fs)

In [4]:
f, ax = plt.subplots(2, figsize=(10, 6), sharex=True)
ax[0].plot(stair_t, stair_acc)
ax[1].plot(stair_t, walk_acc[:stair_t.size])

ax[0].set_title('Stair Ascent')
ax[1].set_title('Walk')

f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# get estimate of gravity
sos = butter(4, 0.8 / fs, output='sos', btype='low')

stair_g = sosfiltfilt(sos, stair_acc, axis=0, padtype='even', padlen=int(fs))
walk_g = sosfiltfilt(sos, walk_acc, axis=0, padtype='even', padlen=int(fs))

ax[0].plot(stair_t, stair_g, '--')
ax[1].plot(stair_t, walk_g[:stair_t.size], '--')

In [6]:
# get the vertical acceleration

stair_vacc = np.sum(stair_acc * stair_g, axis=1)
walk_vacc = np.sum(walk_acc * walk_g, axis=1)

In [8]:
# get the vertical velocity
stair_vvel = cumtrapz(stair_vacc - 1, dx=1/fs, initial=0)
walk_vvel = cumtrapz(walk_vacc - 1, dx=1/fs, initial=0)

f, ax = plt.subplots(figsize=(10, 4))
ax.plot(stair_t, stair_vvel)
ax.plot(stair_t, walk_vvel[:stair_t.size])

ax.plot(stair_t, detrend(stair_vvel) - detrend(stair_vvel)[0], '--', color='C0')
ax.plot(stair_t, detrend(walk_vvel[:stair_t.size]) - detrend(walk_vvel[:stair_t.size])[0], '--', color='C1')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
# get the vertical position
stair_vpos = cumtrapz(stair_vvel, dx=1/fs, initial=0)
walk_vpos = cumtrapz(walk_vvel, dx=1/fs, initial=0)

f, ax = plt.subplots(figsize=(10, 4))
ax.plot(stair_t, stair_vpos)
ax.plot(stair_t, walk_vpos[:stair_t.size])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<br>
<br>

# Testing
<br>
<br>

In [1]:
from scipy.signal import butter, sosfiltfilt, detrend
from scipy.integrate import cumtrapz
import numpy as np
from pathlib import Path
import pandas as pd

from PfyMU.features import *
from PfyMU.gait.train_classifier.core import load_datasets

In [10]:
def vert_pos(x, fs, fcut=0.8):
    sos = butter(4, 2 * fcut / fs, btype='low', output='sos')
    
    # use padtype and a very long padlength to try to smooth out any edge effects of the filter
    g_est = sosfiltfilt(sos, x, axis=1, padtype='even', padlen=int(2 * fs / 3))  # estimate of gravity
    vacc = np.sum(x * g_est, axis=2, keepdims=True)  # keep it 3d
    
    vvel = detrend(cumtrapz(vacc, dx=1/fs, initial=0, axis=1))
    vvel -= vvel[0]
    vpos = cumtrapz(vvel, dx=1/fs, initial=0, axis=1)
    
    return vpos.max(axis=1) - vpos.min(axis=1)

In [4]:
steps = {
    'walking': 0.4,
    'walking-impaired': 0.2,
    'sitting': 900,
    'standing': 300,
    'stairs-ascending': 0.3,
    'stairs-descending': 0.3,
    'cycling-50W': 0.3,
    'cycling-100W': 0.3,
    'default': 1.0
}

In [5]:
# gait_sets_path = Path('/Users/adamol/Documents/Datasets/gait/processed')
gait_sets_path = Path('/home/lukasadamowicz/Documents/Datasets/processed')

datasets = [
    gait_sets_path / 'bluesky2',
    gait_sets_path / 'daliac',
    gait_sets_path / 'ltmm',
    gait_sets_path / 'usc-had'
]

X, Y, subjects, activities = load_datasets(
    datasets, 
    goal_fs=50.0, 
    acc_mag=False, 
    window_length=3.0, 
    window_step=steps
)

# make stair-climbing in the positive class
mask = (activities == 'stairs-ascending') | (activities == 'stairs-descending')
Y_inc_str = Y.copy()
Y_inc_str[mask] = 1

In [11]:
ft_vpos = vert_pos(X, 50.0, fcut=0.8)

In [16]:
data = pd.DataFrame(
    data={
        'Subject': subjects,
        'Activity': activities,
        'Label': Y,
        'Vert Pos': ft_vpos.ravel()
    })

In [13]:
%matplotlib widget
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
f, ax = plt.subplots(figsize=(10, 5))
sns.violinplot(x='Label', y='Vert Pos', data=data, ax=ax)

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …